In [4]:
import numpy as np
import pandas as pd
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [14]:
# fillup the mising values with mean, median
from sklearn.impute import SimpleImputer 

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:5], df.iloc[:,-1], test_size = 0.2 )

In [13]:
X_train
X_test

,age,gender,fever,cough,city
36,38,Female,101.0,Mild,Bangalore
94,79,Male,NaN,Strong,Kolkata
27,33,Female,102.0,Strong,Delhi
80,14,Female,99.0,Mild,Mumbai
19,42,Female,NaN,Strong,Bangalore
29,34,Female,NaN,Strong,Mumbai
64,42,Male,104.0,Mild,Mumbai
51,11,Female,100.0,Strong,Kolkata
3,31,Female,98.0,Mild,Kolkata
16,69,Female,103.0,Mild,Kolkata


In [31]:
# Without COlumn Transformer
# fill the numerical columns(fever) with mean

# fit_transform returns array
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])
# fill the categorical data with most_frequent
# OrdinalEncoder -> cough
oe = OrdinalEncoder(categories = [['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

# OneHotEncoding -> gender, city
ohe = OneHotEncoder(drop = 'first', sparse_output = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

# Age
# with .values we get numpy array, without.values we get dataframe
X_train_age = X_train[['age']].values
X_test_age = X_test[['age']].values

# Now we have to concat the produced columns
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)


In [28]:
X_train_fever.shape
X_train_cough.shape
X_train_gender_city.shape
X_train_age.shape

(80, 1)

In [39]:
# With COlumn Transformer
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers = [
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories = [['Mild', 'Strong']]), ['cough']), 
    ('tnf3', OneHotEncoder(drop = 'first', sparse_output = False), ['gender','city'])
], remainder = 'passthrough')